# Decision Tree Tutorial

To Do:
* Write this cool tutorial
* clean up code a bit
* add pretty graphs

We need to import pandas (a useful data manipulation library), our tree class, and a few other things

In [1]:
from __future__ import division
import pandas as pd
from math import log

import jsonpickle as jp

In [2]:
class TreeNode:
    def __init__(self, label):
        self.label = label
        self.children = {}

        
    def add_child(self, child_path, child_label):
        self.children[child_path] = TreeNode(child_label)

        
    def insert_child(self, child_path, child):
        self.children[child_path] = child

    
    def remove_path(self, child_path):
        del self.children[child_path]

    
    def isLeaf(self):
        return len(self.children) <= 0

    
    def __iter__(self):
        return self.children.__iter__()

    
    def __getitem__(self, key):
        return self.children[key]

    
    def __str__(self):
        return self.label

    
    def traverse(self, feature_vector):
        if self.isLeaf(): 
            return self.label
        next_path = feature_vector[self.label]
        return self.children[next_path].traverse(feature_vector)

    @staticmethod
    def load_tree(save_file):
        with open(save_file, 'r') as f:
            serial_string = f.read()
        return jp.decode(serial_string)

    @staticmethod
    def save_tree(tree_node, save_file):
        serial_string = jp.encode(tree_node)
        with open(save_file, 'w') as f:
            f.write(serial_string)

These are the functions we are going to use. (More details coming later)

In [3]:
def load_data(filename):
    data_array = pd.read_csv(filename)
    return data_array


def b(q):
    if q == 0 or q >= 1:
        # By definition
        return 0
    else:
        return -(q * log(q, 2) + (1 - q) * log(1 - q, 2))
    

    def remain_per_attr(p_k, n_k, p, n):
        remain = ((p_k + n_k) / (p + n)) * b(p_k/(p_k + n_k))
    return remain


def remain_per_attr(p_k, n_k, p, n):
    remain = ((p_k + n_k) / (p + n)) * b(p_k/(p_k + n_k))
    return remain


def remainder(data_array, attribute, goal_attribute):
    remain = 0.0

    for value in data_array[attribute].unique():
        p_k = 0.0
        n_k = 0.0
        p = 0.0
        n = 0.0

        if 'yes' in data_array[data_array[attribute] == value][goal_attribute].value_counts():
            p_k = data_array[data_array[attribute] == value][goal_attribute].value_counts()['yes']

        if 'no' in data_array[data_array[attribute] == value][goal_attribute].value_counts():
            n_k = data_array[data_array[attribute] == value][goal_attribute].value_counts()['no']

        if 'yes' in data_array[goal_attribute].value_counts():
            p = data_array[goal_attribute].value_counts()['yes']

        if 'no' in data_array[goal_attribute].value_counts():
            n = data_array[goal_attribute].value_counts()['no']

        remain += remain_per_attr(p_k, n_k, p, n)

    return remain


def gain(data_array, attribute, goal_attribute):
    # This is always the same, it's based on the overall choices
    p = data_array[goal_attribute].value_counts()['yes']
    n = data_array[goal_attribute].value_counts()['no']
    b_attr = b(p / (n + p))
    remain_attr = remainder(data_array, attribute, goal_attribute)
    return b_attr - remain_attr


def gain_of_attributes(data_array, goal_attribute):
    attribute_gain = {}
    for attr in data_array:
        if not attr == goal_attribute:
            attribute_gain[attr] = gain(data_array, attr, goal_attribute)

    return attribute_gain

def plurality_count(data_array, goal_attribute):
    vdict = (data_array['will_wait'].value_counts()).to_dict()

    return max(vdict, key=vdict.get)

def train_tree(data_array, parent_array, goal_attribute):
    if (len(data_array) <= 0):
        return TreeNode(plurality_count(parent_array, goal_attribute))

    categories = data_array[goal_attribute].unique()
    if (len(categories) == 1):
        return TreeNode(categories[0])

    if (len(data_array.columns) == 1):
        return TreeNode(plurality_count(data_array, goal_attribute)) 

    attribute_gain = gain_of_attributes(data_array, goal_attribute)
    attribute = max(attribute_gain, key=attribute_gain.get)
    tree_node = TreeNode(attribute)
    child_paths = data_array[attribute].unique()

    for path in child_paths:
        child_array = data_array[data_array[attribute] == path]
        child_array = child_array[child_array.columns.difference([attribute])]
        child_tree = train_tree(child_array, parent_array, goal_attribute)
        tree_node.insert_child(path, child_tree)

    return tree_node

def categorize_data(data_array, tree_root):
    output = []
    for data_dict in data_array.to_dict('records'):
        output += [tree_root.traverse(data_dict)]

    return output

In [4]:
my_data = load_data('Data/DecisionTreeData/restaurant_data.csv')
goal_attribute = 'will_wait'

In [5]:
tree_root = train_tree(my_data, None, goal_attribute)
print categorize_data(my_data, tree_root)

['yes', 'no', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'no', 'no', 'no', 'yes']
